## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-11-20-47-15 +0000,nyt,Thiel Gives $3 Million to Group Seeking to Blo...,https://www.nytimes.com/2026/01/11/us/politics...
1,2026-01-11-20-47-00 +0000,nypost,Landlords buzzing about 150 William St. after ...,https://nypost.com/2026/01/11/business/landlor...
2,2026-01-11-20-35-09 +0000,nyt,Why Are Iranians Protesting? What to Know Abou...,https://www.nytimes.com/article/iran-protests-...
3,2026-01-11-20-32-27 +0000,bbc,"Iran warns it will retaliate if US attacks, as...",https://www.bbc.com/news/articles/cm2jek15m8no...
4,2026-01-11-20-28-39 +0000,nyt,Podcasts Hit the Golden Globes: Nominees Inclu...,https://www.nytimes.com/2026/01/11/business/me...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2412/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
37,trump,26
55,ice,17
21,iran,17
107,shooting,10
82,venezuela,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
125,2026-01-11-07-27-11 +0000,nypost,Iran warns US troops and Israel will be target...,https://nypost.com/2026/01/11/world-news/iran-...,62
10,2026-01-11-19-48-53 +0000,nyt,Trump Says Cuba Will No Longer Get ‘Oil or Mon...,https://www.nytimes.com/2026/01/11/us/politics...,61
131,2026-01-11-03-57-21 +0000,nyt,Trump Is Briefed on Options for Striking Iran ...,https://www.nytimes.com/2026/01/10/us/politics...,60
56,2026-01-11-16-28-00 +0000,wsj,South Africa Risks Angering Trump by Hosting N...,https://www.wsj.com/world/africa/south-africa-...,58
55,2026-01-11-16-29-40 +0000,cbc,Trump says no more Venezuelan oil or money to ...,https://www.cbc.ca/news/world/trump-says-no-mo...,56


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
125,62,2026-01-11-07-27-11 +0000,nypost,Iran warns US troops and Israel will be target...,https://nypost.com/2026/01/11/world-news/iran-...
10,57,2026-01-11-19-48-53 +0000,nyt,Trump Says Cuba Will No Longer Get ‘Oil or Mon...,https://www.nytimes.com/2026/01/11/us/politics...
90,52,2026-01-11-11-00-00 +0000,latimes,"Deadly ICE shooting in Minnesota, affordabilit...",https://www.latimes.com/california/story/2026-...
79,28,2026-01-11-13-00-00 +0000,nypost,After Marine vet Trevor Reed was freed from a ...,https://nypost.com/2026/01/11/world-news/marin...
87,27,2026-01-11-11-00-00 +0000,nypost,Bronx building Mamdani used to showcase new ho...,https://nypost.com/2026/01/11/us-news/bronx-bu...
106,26,2026-01-11-10-14-36 +0000,nypost,Neighbor of Illinois surgeon accused of killin...,https://nypost.com/2026/01/11/us-news/neighbor...
104,26,2026-01-11-10-30-00 +0000,wsj,Silicon Valley has long been something of an A...,https://www.wsj.com/politics/policy/its-a-weir...
59,25,2026-01-11-15-55-43 +0000,nypost,Kristi Noem clashes with CNN’s Jake Tapper ove...,https://nypost.com/2026/01/11/us-news/kristi-n...
24,23,2026-01-11-18-56-39 +0000,nypost,"Ilhan Omar urges public to film ICE, chides ag...",https://nypost.com/2026/01/11/us-news/ilhan-om...
34,22,2026-01-11-18-25-23 +0000,nypost,US invasion of Greenland would trigger ‘end of...,https://nypost.com/2026/01/11/us-news/us-invas...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
